In [1]:
import numpy as np

from finn.util.visualization import showInNetron

from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
import finn.core.onnx_exec as oxe
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes

import os
import pandas as pd
import modules.my_metrics as metrics

# Load Original and OOC Model

In [2]:
ori_filename = './models/Best_F1_checked_pytorch.onnx'

# PYNQ-Z1 full build folder
model_folder = './experiments/035_pynq-z1__workspace__w4W2a4__full_build/output_full_build/intermediate_models'
tidy_up_filename = model_folder + '/step_tidy_up.onnx'
finn_filename = model_folder + '/custom_step_streamline.onnx'

### Original Model: cleaned

In [3]:
ori_cleaned_filename = "./models/Best_F1_checked_pytorch_cleaned.onnx"
qonnx_cleanup(ori_filename, out_file= ori_cleaned_filename)

In [4]:
showInNetron(ori_cleaned_filename)

Serving './models/Best_F1_checked_pytorch_cleaned.onnx' at http://0.0.0.0:8083


### Model Wrappers

In [5]:
ori_model = ModelWrapper(ori_cleaned_filename)
tidy_up_model = ModelWrapper(tidy_up_filename)
finn_model = ModelWrapper(finn_filename)

## Execute with Dummy IN

In [6]:
dummy_in = np.random.randint(low = 0, high = 255+1, size = (1, 3, 224, 224)) 
dummy_in = dummy_in.astype(np.float32)

# OOC needs (1, 224, 224, 3)
ooc_dummy_in = np.transpose(dummy_in, (0, 2, 3, 1))

# Print shapes
print(f'dummy_in shape: {dummy_in.shape}')
print(f'dummy_in transposed shape: {ooc_dummy_in.shape}')

dummy_in shape: (1, 3, 224, 224)
dummy_in transposed shape: (1, 224, 224, 3)


### Original Model

In [7]:
input_dict = {"global_in": (dummy_in / 255.) }
output_dict = oxe.execute_onnx(ori_model, input_dict)
produced_ori_qonnx = output_dict[list(output_dict.keys())[0]]
produced_ori_qonnx

array([[-1., -1.]], dtype=float32)

### Tidy Up

In [8]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(tidy_up_model, input_dict)
produced_tidy_up_qonnx = output_dict[list(output_dict.keys())[0]]
produced_tidy_up_qonnx

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(


array([[-1., -1.]], dtype=float32)

In [9]:
# print(output_dict['global_out'][0])
# print(output_dict[list(output_dict.keys())[0]][0])

### FINN Model: streamline

In [10]:
# input_dict = {"global_in": dummy_in} #ooc_dummy_in}
# output_dict = oxe.execute_onnx(finn_model, input_dict)
# produced_tidy_up_qonnx = output_dict[list(output_dict.keys())[0]]
# produced_tidy_up_qonnx

# Eval with DFire MINI

In [11]:
DS_FOLDER = '../../datasets/dfire_mini/'
DS_FOLDER += 'test/'
# DS_FOLDER += 'train/'
DS_IMGS = DS_FOLDER + 'images/'
DS_LABELS = DS_FOLDER + 'labels/'

In [12]:
IMGS_LIST = sorted([DS_IMGS + img_file for img_file in os.listdir(DS_IMGS)])

In [13]:
len(IMGS_LIST)

30

### QONNX model exported in Brevitas after training

In [14]:
# import importlib
# importlib.reload(metrics)

In [15]:
ori_metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = ori_model,
    bipolar=True,
    divide_255=True) # Ori model does not have the preprocess included

Number of test samples: 30

Image file: ../../datasets/dfire_mini/test/images/AoF06872.jpg: 
	label [0. 0.]
	pred [0. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF07484.jpg: 
	label [0. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF07856.jpg: 
	label [0. 1.]
	pred [0. 1.]
Image file: ../../datasets/dfire_mini/test/images/AoF07878.jpg: 
	label [0. 1.]
	pred [0. 1.]
Image file: ../../datasets/dfire_mini/test/images/AoF08019.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF08122.jpg: 
	label [1. 0.]
	pred [0. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF08143.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF08281.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF08304.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/PublicDataset01109.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../data

In [16]:
ori_df = pd.DataFrame(ori_metrics)
ori_df

,Smoke,Fire,Mean
Accuracy,0.9000,1.0,0.9500
Precision,0.9524,1.0,0.9762
Recall,0.9091,1.0,0.9546
F1,0.9302,1.0,0.9651


### Tidy Up Model

In [17]:
clean_metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = tidy_up_model,
    bipolar=True,
    divide_255=False) # Tidy Up model has the preprocess included

Number of test samples: 30



/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(


Image file: ../../datasets/dfire_mini/test/images/AoF06872.jpg: 
	label [0. 0.]
	pred [0. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF07484.jpg: 
	label [0. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF07856.jpg: 
	label [0. 1.]
	pred [0. 1.]
Image file: ../../datasets/dfire_mini/test/images/AoF07878.jpg: 
	label [0. 1.]
	pred [0. 1.]
Image file: ../../datasets/dfire_mini/test/images/AoF08019.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF08122.jpg: 
	label [1. 0.]
	pred [0. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF08143.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF08281.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/AoF08304.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/PublicDataset01109.jpg: 
	label [1. 0.]
	pred [1. 0.]
Image file: ../../datasets/dfire_mini/test/images/

In [18]:
print(clean_metrics)

{'Smoke': {'Accuracy': 0.9, 'Precision': 0.9524, 'Recall': 0.9091, 'F1': 0.9302}, 'Fire': {'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'F1': 1.0}, 'Mean': {'Accuracy': 0.95, 'Precision': 0.9762, 'Recall': 0.9546, 'F1': 0.9651}}


In [19]:
clean_df = pd.DataFrame(clean_metrics)
clean_df

,Smoke,Fire,Mean
Accuracy,0.9000,1.0,0.9500
Precision,0.9524,1.0,0.9762
Recall,0.9091,1.0,0.9546
F1,0.9302,1.0,0.9651


In [22]:
ori_df

,Smoke,Fire,Mean
Accuracy,0.9000,1.0,0.9500
Precision,0.9524,1.0,0.9762
Recall,0.9091,1.0,0.9546
F1,0.9302,1.0,0.9651


### FINN Model

In [20]:
# finn_metrics = metrics.eval_quant_onnx(
#     imgs_list = IMGS_LIST,
#     labels_dir = DS_LABELS,
#     model_wrapped = ooc_model,
#     bipolar=True,
#     divide_255=False)

In [21]:
# finn_df = pd.DataFrame(finn_metrics)
# finn_df